In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocess as mp
import time
from helpers.GPTS_Kernels import RBFKernel
import helpers.GP_opt_sims as GP

In [43]:
def GP_sim_wrapper(info, print_upd = True): 
    import numpy as np
    from helpers.GPTS_Kernels import RBFKernel
    import helpers.GP_opt_sims as GP
    import pandas as pd
    
    def genFunc(func_info, beta):
        if(func_info[0] == 0):
            _, m, o, l = func_info
            truef = lambda x: o * np.exp(-(x-m)*(x-m)/l) - (o * np.sqrt(3.14159 * l))
            return GP.GPFunction(truef, 1/beta, [0, 1], [m, o - (o * np.sqrt(3.14159 * l))])
    
    idnum, task = info
    T, func_info, GPR_type, data_amt, data_dist, num_samples, gpr_data = task
    len_scale, beta = gpr_data

    kernel = RBFKernel(len_scale)
    func = genFunc(func_info, beta)

    PERC_FREQ = 0.1
    print_intvl = int(num_samples*PERC_FREQ/100)
    print_intvl = print_intvl if print_intvl > 0 else 1

    regret_vecs = np.zeros((num_samples, T))
    NUM_BUCKETS = 50
    data_rem = np.zeros(NUM_BUCKETS)
    buckets = np.linspace(func.x_min, func.x_max, NUM_BUCKETS + 1)
    for i in range(num_samples):
        #generate data: TODO-non uniform
        data_X = np.random.uniform(func.x_min, func.x_max, size = (data_amt, 1))
        data_Y = np.array([func.sample(p) for p in data_X])

        if(GPR_type[0] == 0): #full-start
            rew = GP.FS(T, func, data_X, data_Y, kernel, beta)
        elif(GPR_type[0] == 1): #level set
            rew, data_left = GP.AR_lvl_set(T, func, data_X, data_Y, kernel, beta, GPR_type[1])
            hist, _ = np.histogram(data_left, buckets)
            data_rem = data_rem + hist
        elif(GPR_type[0] == 2): #fixed data radius
            rew, data_left = GP.AR_rad(T, func, data_X, data_Y, kernel, beta, GPR_type[1])
            hist, _ = np.histogram(data_left, buckets)
            data_rem = data_rem + hist
        else:
            pass      
    
        regret_vecs[i] = np.cumsum(func.opt - np.array(rew))
        
        if(print_upd and ((i+1)%(print_intvl) == 0)):
            whitespace = " " * 20
            print("Task #" +str(idnum) + ":" + str(task) + " is " + str((i+1)/num_samples*100) + "% completed." + whitespace , end='\r')
    
    avgs = np.mean(regret_vecs, axis=0)
    lower_CBs = avgs-2*np.std(regret_vecs,axis=0)/np.sqrt(num_samples)
    upper_CBs = avgs+2*np.std(regret_vecs,axis=0)/np.sqrt(num_samples)

    all_params = [str(func_info), GPR_type[0], GPR_type[1], data_amt, data_dist, num_samples, len_scale, beta]
    df_row = all_params + list(avgs) + list(lower_CBs) + list(upper_CBs)

    df_cols = ["func", "alg_type", "alg_param", "DataAmt", "DataDist", "num_samples", "len_scale", "beta"]

    if(GPR_type[0] > 0):
        data_cols = df_cols + list(range(NUM_BUCKETS))
        data_rem /= num_samples
        data_row = all_params + list(data_rem)
        data_df = pd.DataFrame(columns = data_cols)
        data_df.loc[0] = data_row
        data_df.to_csv("temp/ar_data_task"+str(idnum)+".csv")

    c = list(range(1, T + 1))
    df_cols = df_cols + c + ["low_"+str(n) for n in c] + ["high_"+str(n) for n in c]

    df= pd.DataFrame(columns = df_cols)
    df.loc[0] = df_row
    df.to_csv("temp/task"+str(idnum)+".csv")
    return 1

In [4]:
#fixed hyperparams
beta = 5
len_scale = 50

In [5]:
#simulation params
T = 500 #simulation horizon
num_samples = 1000
#different ways of data distribution across sample space
data_dists = [0] #0-uniform,
functions = [[0, 0.5, 2, 1/50]] #0-single peak, 1-bimodal, 2-?
data_amt_list = np.arange(30, 130, 30) #total number of data points (16)
AR_types = [[0, 0]] + [[1, x] for x in [0.5, 1, 2, 4]] #0-FS, 1- AR level set, 2- AR bandwidth

#generate tasks
tasks = [[T, func, AR_type, data_amt, data_dist, num_samples, [len_scale, beta]] 
         for func in functions for AR_type in AR_types for data_amt in data_amt_list for data_dist in data_dists]
info = [(i, tasks[i]) for i in range(len(tasks))]
len(info)

20

In [46]:
start = time.time()
p = mp.Pool(12)
results = p.map(GP_sim_wrapper, info)
print()
print(time.time()-start)


15923.15447640419


In [6]:
T=500
df_cols = ["func", "alg_type", "alg_param", "DataAmt", "DataDist", "num_samples", "len_scale", "beta"]
c = list(range(1, T + 1))
strc = [str(n) for n in c]
NUM_BUCKETS = 50
bucket_ids = [str(n) for n in range(NUM_BUCKETS)]
data_cols = df_cols + bucket_ids
df_cols = df_cols + strc + ["low_"+n for n in strc] + ["high_"+n for n in strc]

gp_df= pd.DataFrame(columns = df_cols)
ar_data_df= pd.DataFrame(columns = data_cols)

#for idnum, _ in info:
for idnum in range(len(info)):
    df_task = pd.read_csv("temp/task"+str(idnum)+".csv", index_col=0)
    gp_df.loc[idnum] = df_task.loc[0]
    if(df_task.loc[0, "alg_type"] > 0):
        ar_df_task = pd.read_csv("temp/ar_data_task"+str(idnum)+".csv", index_col=0)
        ar_data_df.loc[len(ar_data_df)] = ar_df_task.loc[0]

gp_df.to_csv("GP_df.csv")
ar_data_df.to_csv("GP_ar_data.csv")

In [7]:
ar_data_df

,func,alg_type,alg_param,DataAmt,DataDist,num_samples,len_scale,beta,0,1,...,40,41,42,43,44,45,46,47,48,49
0,"[0, 0.5, 2, 0.02]",1,0.5,30,0,1000,50,5,0.269,0.376,...,0.455,0.432,0.442,0.457,0.44,0.419,0.418,0.389,0.331,0.262
1,"[0, 0.5, 2, 0.02]",1,0.5,60,0,1000,50,5,0.786,0.91,...,0.971,1.02,0.962,1.027,1.015,1.011,1.017,0.99,0.93,0.764
2,"[0, 0.5, 2, 0.02]",1,0.5,90,0,1000,50,5,1.3,1.507,...,1.727,1.658,1.616,1.691,1.621,1.626,1.602,1.521,1.493,1.297
3,"[0, 0.5, 2, 0.02]",1,0.5,120,0,1000,50,5,1.955,2.048,...,2.26,2.239,2.213,2.257,2.304,2.203,2.26,2.159,2.103,1.854
4,"[0, 0.5, 2, 0.02]",1,1,30,0,1000,50,5,0.274,0.332,...,0.418,0.387,0.385,0.415,0.45,0.411,0.418,0.359,0.315,0.26
5,"[0, 0.5, 2, 0.02]",1,1,60,0,1000,50,5,0.747,0.85,...,1.083,0.959,1.026,1.044,1.025,1.008,1.014,0.935,0.837,0.749
6,"[0, 0.5, 2, 0.02]",1,1,90,0,1000,50,5,1.363,1.471,...,1.562,1.559,1.639,1.635,1.643,1.566,1.658,1.574,1.442,1.349
7,"[0, 0.5, 2, 0.02]",1,1,120,0,1000,50,5,1.969,1.981,...,2.214,2.262,2.218,2.212,2.272,2.262,2.248,2.1,2.076,1.983
8,"[0, 0.5, 2, 0.02]",1,2,30,0,1000,50,5,0.25,0.299,...,0.383,0.383,0.387,0.392,0.371,0.365,0.336,0.307,0.278,0.272
9,"[0, 0.5, 2, 0.02]",1,2,60,0,1000,50,5,0.762,0.826,...,1.004,0.997,0.946,0.945,0.982,0.952,0.981,0.884,0.832,0.696
